<a href="https://colab.research.google.com/github/amitadhainje/CodingAssignment-UBS/blob/master/Bert_SentimentAnalysis_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#Installing the required modules
!pip install bert-for-tf2
!pip install sentencepiece
print ("The modules like bert-for-tf2 and sentencepiece is installed")

The modules like bert-for-tf2 and sentencepiece is installed


In [0]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [23]:
print ("The tensorflow version - ",tf.__version__)
print ("The tensorflow_hub  version - ",hub.__version__)
print ("The pandas version - ",pd.__version__)
print ("The numpy version - ",np.__version__)


The tensorflow version -  2.1.0
The tensorflow_hub  version -  0.7.0
The pandas version -  0.25.3
The numpy version -  1.18.2


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
columns = ['sentiment', 'id', 'date', 'query','user', 'text']
trainData = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train.csv", header=None, names=columns,
                   engine="python", encoding='latin1')

#positiveData = dataset[dataset['sentiment'] == 4][0:50000]
#negativeData = dataset[dataset['sentiment'] == 0][0:50000]
#frames = [positiveData, negativeData]
#trainData = pd.concat(frames)
trainData.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
trainData.drop(['id','date','query','user'], axis=1, inplace=True)
trainData.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [27]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, 'lxml').get_text()
    tweet = re.sub(r"@[A-Z-a-z-0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Z-a-z-0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z0-9.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

data_clean = [clean_tweet(tweet) for tweet in trainData.text]
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

In [0]:
data_labels = trainData.sentiment.values
data_labels[data_labels == 4] = 1

In [0]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1', trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [0]:
def encode_sentence(sent):
    return ['[CLS]'] + tokenizer.tokenize(sent) + ['[SEP]']

data_inputs = [encode_sentence(sentence) for sentence in data_clean]

1. Add "CLS" and "SEP" tokens to the starting and end of the sequence. Here we are assuming that there is only one sequence.

2. A "BERT_LAYER" takes 3 different inputs.

3. First is the tokenized version of the input sentences i.e. the one we have created in using the above "encode_sentence" function. Second is the list of the masks or "input_masks" which helps bert to differentiate between the actual values and the padding values. It tells the bert where are the padding values for a given sentence. The padded tokens should not be used while processing. Third one is the segment ids , which will be used if there are multiple sentences in a particular records. Segment ids are combination of '0's and '1's and they tell the bert which is the first sentence and which one is the second sentence in one particular record.

4. While initializing the "bert_layer" , we mention "trainable=False" , we want to use BERT as it is and we do not want to fine-tune it.

In [0]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id
    return seg_ids





In [0]:
# The training data is sorted according to the target variable, hence we are going to shuffle the data
import random
data_with_len = [[sent,data_labels[i], len(sent)]
                 for i,sent in enumerate(data_inputs)]

random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]), 
               get_mask(sent_lab[0]),
                get_segments(sent_lab[0]) 
               ], sent_lab[1])   
                for sent_lab in data_with_len if sent_lab[2] > 7 ]

In [0]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE )
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [43]:
sampleSent = ["[CLS]"] + tokenizer.tokenize("Roses are red") + ["[SEP]"]
print (sampleSent)
bert_layer([tf.expand_dims(tf.cast(get_ids(sampleSent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(sampleSent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(sampleSent), tf.int32), 0)])

['[CLS]', 'Roses', 'are', 'red', '[SEP]']


[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-0.7411164 ,  0.4618967 ,  0.99993765, -0.99560326,  0.96021503,
          0.68910116,  0.9868691 , -0.9526616 , -0.9781783 , -0.68281555,
          0.99076134,  0.99894404, -0.993728  , -0.99988776,  0.65058947,
         -0.982285  ,  0.9928449 , -0.58514524, -0.99998057, -0.78017884,
         -0.20368141, -0.99992603,  0.31423122,  0.93991727,  0.984753  ,
          0.11941554,  0.993232  ,  0.99997103,  0.88947356,  0.22935857,
          0.36462295, -0.99528813,  0.83558977, -0.99943525,  0.17936641,
          0.02297609,  0.3446346 , -0.29723346,  0.77074224, -0.84347856,
         -0.7860427 , -0.46762857,  0.53143156, -0.52513546,  0.7981314 ,
          0.37057903,  0.2312386 , -0.03575641, -0.24646711,  0.99996597,
         -0.9804097 ,  0.9999313 , -0.9756166 ,  0.9991564 ,  0.9958855 ,
          0.60811   ,  0.9970868 ,  0.24700336, -0.99403846,  0.5302822 ,
          0.9865914 ,  0.03325732,  0.9593753 , -0.14395088, 

In the above output, there are two tensors which are displayed for the "sampleSent". The first tensor corresponds to the "CLS" element which will be used for the classification task. The shape is (1,768) where "1" is the batch i.e we have given only one input and "768" is the embedding dimension. 

The second tensor is of shape (1,5,768). Again "1" is for the batch , "5" as we have total 5 tokens in our sample sentence and 768 is the embedding size.

tf.cast = converts the list into a tensor.<br />
tf.expand_dims = Returns a tensor with an additional dimension inserted at index axis.


In [0]:
class MyCustomCallBack(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print ("Checkpoint is saved at {} ". format(checkpoint_path))

In [0]:
class DCNNBertEmbedding(tf.keras.Model):

    def __init__(self,
                 nb_filters=50,# 50 feature dectectors
                 FFN_units=512, # number of hidden units in last 2 layers
                 nb_classes=2,
                 dropout_rate = 0.2,
                 name = "dcnn"):
        super(DCNNBertEmbedding, self).__init__(name=name)

        self.bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1', trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        
        self.trigram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=3,
                                    padding="valid",
                                    activation="relu")
        
        self.fourgram = layers.Conv1D(filters=nb_filters, 
                                    kernel_size=4,
                                    padding="valid",
                                    activation="relu")
        
        self.pool = layers.GlobalMaxPooling1D()

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")

        self.dropout = layers.Dropout(rate=dropout_rate)

        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1, activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes, activation="softmax")

    # The above function is similar to the regular embedding layer.
    # "_" = returns the embedding for the "CLS" token 
    # embs = returns the embeddings for all the words.
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([
                                   all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :],
                                ])
        return embs

    def call(self, inputs, training):

        x = self.embed_with_bert(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)

        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)

        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # batch_size = 3*nb_filters
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output


In [0]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 5

dcnn = DCNNBertEmbedding(
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            dropout_rate=DROPOUT_RATE)

if NB_CLASSES == 2:
    dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=['accuracy'])
else:
    dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=['sparse_categorical_accuracy'])
    
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/ckpt_bert_embedding"

ckpt = tf.train.Checkpoint(dcnn=dcnn)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ("Latest checkpoint is restored")

In [0]:
dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallBack()])


Epoch 1/5
    909/Unknown - 1037s 1s/step - loss: 0.4552 - accuracy: 0.7880